In [2]:
from netCDF4 import Dataset
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import numpy as np
import os

In [3]:
from netCDF4 import Dataset

# Create a new NetCDF file
nc_file = Dataset('example_polar_stereographic.nc', 'w', format='NETCDF4_CLASSIC')

# Create a new variable for the projection
proj_var = nc_file.createVariable('polar_stereographic', 'i4')

# Assign the attributes to the variable
proj_var.long_name = 'Polar Stereographic Projection'
proj_var.grid_mapping_name = 'polar_stereographic'
proj_var.straight_vertical_longitude_from_pole = 0.0
proj_var.latitude_of_projection_origin = 90.0
proj_var.standard_parallel = 70.0
proj_var.false_easting = 0.0
proj_var.false_northing = 0.0
proj_var.semi_major_axis = 6378137.0
proj_var.semi_minor_axis = 6356752.31414
proj_var.inverse_flattening = 298.2572221

# Close the file
nc_file.close()

print("NetCDF file 'example_polar_stereographic.nc' created successfully.")


NetCDF file 'example_polar_stereographic.nc' created successfully.


In [3]:
def lat_lon_reproj(nc_folder, nc_indx):
    os.chdir(nc_folder)
    full_direc = os.listdir()
    nc_files = [ii for ii in full_direc if ii.endswith('.nc')]
    g16_data_file = nc_files[nc_indx] # select .nc file
    print(nc_files[nc_indx]) # print file name

    # designate dataset
    g16nc = Dataset(g16_data_file, 'r')
    var_names = [ii for ii in g16nc.variables]
    var_name = var_names[0]
    try:
        band_id = g16nc.variables['band_id'][:]
        band_id = ' (Band: {},'.format(band_id[0])
        band_wavelength = g16nc.variables['band_wavelength']
        band_wavelength_units = band_wavelength.units
        band_wavelength_units = '{})'.format(band_wavelength_units)
        band_wavelength = ' {0:.2f} '.format(band_wavelength[:][0])
        print('Band ID: {}'.format(band_id))
        print('Band Wavelength: {} {}'.format(band_wavelength,band_wavelength_units))
    except:
        band_id = ''
        band_wavelength = ''
        band_wavelength_units = ''

    # Non-geostationary projection info and retrieving relevant constants
    proj_info = g16nc.variables['polar_stereographic']
    lon_origin = proj_info.straight_vertical_longitude_from_pole
    lat_origin = proj_info.latitude_of_projection_origin
    std_parallel = proj_info.standard_parallel
    false_easting = proj_info.false_easting
    false_northing = proj_info.false_northing
    r_eq = proj_info.semi_major_axis
    r_pol = proj_info.semi_minor_axis

    # grid info
    x = g16nc.variables['x'][:]
    y = g16nc.variables['y'][:]

    # data info    
    data = g16nc.variables[var_name][:]
    data_units = g16nc.variables[var_name].units
    data_time_grab = ((g16nc.time_coverage_end).replace('T',' ')).replace('Z','')
    data_long_name = g16nc.variables[var_name].long_name
    
    # close file when finished
    g16nc.close()
    g16nc = None

    # create meshgrid filled with x and y coordinates
    X, Y = np.meshgrid(x, y)

    # lat/lon calc routine for polar stereographic projection
    # Formula reference: https://proj.org/operations/projections/stere.html
    t = np.sqrt(X**2 + Y**2)
    theta = np.arctan2(X, -Y)
    lat = 90 - 2 * np.arctan(t / (2 * r_eq)) * (180.0 / np.pi)
    lon = (theta * (180.0 / np.pi) + lon_origin) % 360

    # print test coordinates
    print('{} N, {} W'.format(lat[318,1849], abs(lon[318,1849])))

    return lon, lat, data, data_units, data_time_grab, data_long_name, band_id, band_wavelength, band_wavelength_units, var_name

nc_folder = './rad_nc_files/' # define folder where .nc files are located

file_indx = 6

# main data grab from function above
lon, lat, data, data_units, data_time_grab, data_long_name, band_id, band_wavelength, band_units, var_name = lat_lon_reproj(nc_folder, file_indx)

bbox = [np.min(lon), np.min(lat), np.max(lon), np.max(lat)] # set bounds for plotting

# figure routine for visualization
fig = plt.figure(figsize=(9, 4), dpi=200)

n_add = 0
m = Basemap(llcrnrlon=bbox[0] - n_add, llcrnrlat=bbox[1] - n_add, urcrnrlon=bbox[2] + n_add, urcrnrlat=bbox[3] + n_add, resolution='i', projection='cyl')
m.drawcoastlines(linewidth=0.5)
m.drawcountries(linewidth=0.25)
m.pcolormesh(lon, lat, data, latlon=True)
parallels = (np.linspace(np.min(lat), np.max(lat), 5))
m.drawparallels(parallels, labels=[True, False, False, False])
meridians = np.linspace(np.min(lon), np.max(lon), 5)
m.drawmeridians(meridians, labels=[False, False, False, True])
cb = m.colorbar()

data_units = ((data_units.replace('-', '^{-')).replace('1', '1}')).replace('2', '2}')
plt.rc('text', usetex=True)
cb.set_label(r'%s $ \left[ \mathrm{%s} \right] $' % (var_name, data_units))
plt.title('{0}{2}{3}{4} on {1}'.format(data_long_name, data_time_grab, band_id, band_wavelength, band_units))
plt.tight_layout()

plt.savefig('non_geo_demo.png', dpi=200)
plt.show()


example_polar_stereographic.nc


KeyError: 'x'